In [46]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import mediapipe as mp
import pickle
import json
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [47]:
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
sequence_length = 30
min_sequences_per_class = 10
DATA_PATH = '/home/smayan/Desktop/Cricket Pose Estimation /Data'
PROCESSED_DATA_PATH = '/home/smayan/Desktop/Cricket Pose Estimation /ProcessedData'

In [48]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, results

In [49]:
def draw_styled_landmarks(image, results):
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

In [50]:
def extract_keypoints(results):
    return np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

In [51]:
def check_data_structure():
    """Check and print data structure"""
    print("Checking data structure...")
    if not os.path.exists(DATA_PATH):
        print(f"ERROR: {DATA_PATH} does not exist!")
        return False
    
    actions = [folder for folder in os.listdir(DATA_PATH) 
               if os.path.isdir(os.path.join(DATA_PATH, folder))]
    
    if not actions:
        print(f"ERROR: No action folders found in {DATA_PATH}")
        return False
    
    total_videos = 0
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        videos = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        print(f"{action}: {len(videos)} videos")
        total_videos += len(videos)
    
    print(f"Total videos found: {total_videos}")
    
    if total_videos == 0:
        print("ERROR: No video files found!")
        return False
    
    return True

In [52]:
def create_processed_data_structure():
    """Create directory structure for processed data"""
    if not os.path.exists(PROCESSED_DATA_PATH):
        os.makedirs(PROCESSED_DATA_PATH)
    
    sequences_dir = os.path.join(PROCESSED_DATA_PATH, 'sequences')
    metadata_dir = os.path.join(PROCESSED_DATA_PATH, 'metadata')
    
    if not os.path.exists(sequences_dir):
        os.makedirs(sequences_dir)
    if not os.path.exists(metadata_dir):
        os.makedirs(metadata_dir)
    
    return sequences_dir, metadata_dir


In [53]:
def save_processing_metadata(actions, video_counts, total_sequences, sequences_dir, metadata_dir):
    """Save metadata about the processing"""
    metadata = {
        'processing_date': datetime.now().isoformat(),
        'sequence_length': sequence_length,
        'min_sequences_per_class': min_sequences_per_class,
        'actions': actions.tolist(),
        'video_counts': video_counts,
        'total_sequences_per_class': total_sequences,
        'keypoint_dimensions': 132,  # 33 pose landmarks × 4 values
        'data_path': DATA_PATH,
        'processed_data_path': PROCESSED_DATA_PATH
    }
    
    # Save as JSON for human readability
    metadata_file = os.path.join(metadata_dir, 'processing_metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)
    
    print(f"Metadata saved to: {metadata_file}")
    return metadata

In [54]:
def save_sequences_and_labels(sequences, labels, label_map, sequences_dir, metadata_dir):
    """Save sequences and labels to disk"""
    print("\nSaving processed sequences...")
    
    # Save sequences as numpy array
    sequences_file = os.path.join(sequences_dir, 'sequences.npy')
    np.save(sequences_file, sequences)
    print(f"Sequences saved to: {sequences_file}")
    print(f"Sequences shape: {sequences.shape}")
    
    # Save labels as numpy array
    labels_file = os.path.join(sequences_dir, 'labels.npy')
    np.save(labels_file, labels)
    print(f"Labels saved to: {labels_file}")
    print(f"Labels shape: {labels.shape}")
    
    # Save label mapping
    label_map_file = os.path.join(metadata_dir, 'label_map.pkl')
    with open(label_map_file, 'wb') as f:
        pickle.dump(label_map, f)
    print(f"Label map saved to: {label_map_file}")
    
    # Also save label map as JSON for readability
    label_map_json = os.path.join(metadata_dir, 'label_map.json')
    with open(label_map_json, 'w') as f:
        json.dump(label_map, f, indent=4)
    
    # Calculate and save file sizes
    sequences_size = os.path.getsize(sequences_file) / (1024 * 1024)  # MB
    labels_size = os.path.getsize(labels_file) / (1024 * 1024)  # MB
    
    print(f"\nFile sizes:")
    print(f"Sequences: {sequences_size:.2f} MB")
    print(f"Labels: {labels_size:.2f} MB")
    print(f"Total: {sequences_size + labels_size:.2f} MB")

In [55]:
def load_processed_sequences():
    """Load previously processed sequences"""
    sequences_dir = os.path.join(PROCESSED_DATA_PATH, 'sequences')
    metadata_dir = os.path.join(PROCESSED_DATA_PATH, 'metadata')
    
    # Check if processed data exists
    sequences_file = os.path.join(sequences_dir, 'sequences.npy')
    labels_file = os.path.join(sequences_dir, 'labels.npy')
    label_map_file = os.path.join(metadata_dir, 'label_map.pkl')
    metadata_file = os.path.join(metadata_dir, 'processing_metadata.json')
    
    if not all(os.path.exists(f) for f in [sequences_file, labels_file, label_map_file, metadata_file]):
        print("Processed data not found. Will process videos from scratch.")
        return None, None, None, None
    
    print("Loading previously processed sequences...")
    
    try:
        # Load sequences and labels
        sequences = np.load(sequences_file, allow_pickle=True)
        labels = np.load(labels_file, allow_pickle=True)
        
        # Load label map
        with open(label_map_file, 'rb') as f:
            label_map = pickle.load(f)
        
        # Load metadata
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        
        print(f"Loaded sequences shape: {sequences.shape}")
        print(f"Loaded labels shape: {labels.shape}")
        print(f"Processing date: {metadata['processing_date']}")
        print(f"Actions: {metadata['actions']}")
        
        return sequences, labels, label_map, metadata
    except Exception as e:
        print(f"Error loading processed data: {e}")
        print("Will process videos from scratch.")
        return None, None, None, None

In [56]:
def load_data():
    actions = np.array(sorted([folder for folder in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH,folder))]))

    print(f"Detected Cricket Shots:  {actions}")

    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        print(f"{action}: {len(video_files)} videos")

    return actions

In [57]:
actions = load_data()

Detected Cricket Shots:  ['Backfoot punch' 'Cover drive' 'Cut Shot' 'FBD' 'Flick'
 'Front Food defence' 'On Drive' 'Pull Shot' 'Reverse Sweep'
 'Straight Drive' 'Sweep' 'Uppercut' 'loft']
Backfoot punch: 19 videos
Cover drive: 29 videos
Cut Shot: 43 videos
FBD: 15 videos
Flick: 22 videos
Front Food defence: 32 videos
On Drive: 40 videos
Pull Shot: 40 videos
Reverse Sweep: 30 videos
Straight Drive: 25 videos
Sweep: 27 videos
Uppercut: 29 videos
loft: 31 videos


In [58]:
def extract_sequences_from_videos(actions, force_reprocess=False):
    """Extract sequences from videos with save/load functionality"""
    
    # Try to load existing processed data first
    if not force_reprocess:
        sequences, labels, label_map, metadata = load_processed_sequences()
        if sequences is not None:
            print("Using previously processed sequences.")
            return sequences, labels, label_map
    
    print("Processing videos to extract sequences...")
    
    # Create directory structure
    sequences_dir, metadata_dir = create_processed_data_structure()
    
    sequences = []
    labels = []
    label_map = {label: num for num, label in enumerate(actions)}
    video_counts = {}
    total_sequences = {}
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for action in actions:
            print(f"\nProcessing {action} videos...")
            action_path = os.path.join(DATA_PATH, action)
            
            if not os.path.exists(action_path):
                print(f"Warning: Action path {action_path} does not exist. Skipping...")
                continue
            
            video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
            video_counts[action] = len(video_files)
            
            if len(video_files) == 0:
                print(f"Warning: No video files found for action {action}")
                continue
            
            action_sequences = []
            
            for video_idx, video_file in enumerate(video_files):
                print(f"  Processing video {video_idx + 1}/{len(video_files)}: {video_file}")
                video_path = os.path.join(action_path, video_file)
                
                try:
                    cap = cv.VideoCapture(video_path)
                    
                    if not cap.isOpened():
                        print(f"    Error: Could not open video {video_file}")
                        continue
                    
                    # Get total frames in video
                    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
                    if total_frames < sequence_length:
                        print(f"    Warning: Video too short ({total_frames} frames), skipping...")
                        cap.release()
                        continue
                    
                    # Extract multiple sequences using sliding window
                    stride = max(1, sequence_length // 4)  # 75% overlap
                    video_sequences = 0
                    
                    for start_frame in range(0, total_frames - sequence_length + 1, stride):
                        cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
                        sequence = []
                        
                        for frame_idx in range(sequence_length):
                            ret, frame = cap.read()
                            if not ret:
                                break
                            
                            # Resize frame for consistency
                            frame = cv.resize(frame, (640, 480))
                            
                            _, results = mediapipe_detection(frame, holistic)
                            keypoints = extract_keypoints(results)
                            sequence.append(keypoints)
                        
                        if len(sequence) == sequence_length:
                            action_sequences.append(sequence)
                            video_sequences += 1
                    
                    cap.release()
                    print(f"    Extracted {video_sequences} sequences from {video_file}")
                    
                except Exception as e:
                    print(f"    Error processing video {video_file}: {e}")
                    continue
            
            # Data augmentation for classes with fewer sequences
            original_count = len(action_sequences)
            while len(action_sequences) < min_sequences_per_class and original_count > 0:
                # Simple augmentation: add noise
                original_seq = np.array(action_sequences[len(action_sequences) % original_count])
                noise = np.random.normal(0, 0.01, original_seq.shape)
                augmented_seq = original_seq + noise
                action_sequences.append(augmented_seq.tolist())
            
            augmented_count = len(action_sequences) - original_count
            if augmented_count > 0:
                print(f"  Added {augmented_count} augmented sequences")
            
            # Add sequences and labels
            for seq in action_sequences:
                sequences.append(seq)
                labels.append(label_map[action])
            
            total_sequences[action] = len(action_sequences)
            print(f"  Total sequences for {action}: {len(action_sequences)}")
    
    # Check if we have any sequences
    if len(sequences) == 0:
        raise ValueError("No sequences were extracted! Check your video files and ensure they are readable.")
    
    # Convert to numpy arrays
    sequences = np.stack(sequences)  # (num_samples, sequence_length, num_features)
    labels = np.array(labels)
    
    print(f"\nFinal dataset:")
    print(f"Total sequences: {len(sequences)}")
    print(f"Sequence shape: {sequences.shape}")
    
    # Save processed data
    save_sequences_and_labels(sequences, labels, label_map, sequences_dir, metadata_dir)
    save_processing_metadata(actions, video_counts, total_sequences, sequences_dir, metadata_dir)
    
    return sequences, labels, label_map

In [59]:
def load_cricket_data():
    """Load cricket shot data with variable number of videos per class"""
    if not check_data_structure():
        raise FileNotFoundError(f"Data validation failed for {DATA_PATH}")
    
    actions = np.array(sorted([folder for folder in os.listdir(DATA_PATH) 
                              if os.path.isdir(os.path.join(DATA_PATH, folder))]))
    
    if len(actions) == 0:
        raise ValueError(f"No action folders found in {DATA_PATH}")
    
    print(f"Detected cricket shots: {actions}")
    
    # Count videos per class
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        print(f"{action}: {len(video_files)} videos")
    
    return actions

In [60]:
def create_hybrid_cnn_lstm_model(input_shape, num_classes):
    """Create a Hybrid CNN-LSTM model for cricket pose estimation"""
    model = Sequential()
    
    # CNN layers for spatial feature extraction
    model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'), 
                             input_shape=input_shape))
    model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv1D(128, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Conv1D(128, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Dropout(0.25)))
    
    # Flatten the CNN output for LSTM
    model.add(TimeDistributed(Flatten()))
    
    # LSTM layers for temporal sequence modeling
    model.add(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
    model.add(LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.3))
    
    # Dense layers for classification
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [61]:
def train_cricket_model(force_reprocess=False):
    """Main training pipeline with save/load functionality"""
    
    try:
        # Load data
        actions = load_cricket_data()
        
        # Extract or load sequences
        print("\n" + "="*50)
        print("SEQUENCE EXTRACTION/LOADING")
        print("="*50)
        
        X, y, label_map = extract_sequences_from_videos(actions, force_reprocess=force_reprocess)
        
        # DEBUG INFO
        print(f"\nDEBUG INFO:")
        print(f"X type: {type(X)}")
        print(f"X shape: {X.shape}")
        print(f"X ndim: {X.ndim}")
        print(f"y type: {type(y)}")
        print(f"y shape: {y.shape}")
        
        # Validation checks
        if X.size == 0:
            raise ValueError("No sequences were extracted! Check your video files and data path.")
        
        if X.ndim != 3:
            raise ValueError(f"Expected X to have 3 dimensions (samples, timesteps, features), got {X.ndim} dimensions with shape {X.shape}")
        
        if len(X) != len(y):
            raise ValueError(f"Mismatch between sequences ({len(X)}) and labels ({len(y)})")
        
        print(f"\nDataset summary:")
        print(f"Dataset shape: {X.shape}")
        print(f"Labels shape: {y.shape}")
        print(f"Classes: {list(label_map.keys())}")
        
        # Print class distribution
        unique, counts = np.unique(y, return_counts=True)
        print(f"\nClass distribution:")
        for class_idx, count in zip(unique, counts):
            class_name = [k for k, v in label_map.items() if v == class_idx][0]
            print(f"  {class_name}: {count} sequences")
        
        # Check if we have enough data for training
        if len(unique) < 2:
            raise ValueError("Need at least 2 classes for training")
        
        min_class_size = min(counts)
        if min_class_size < 2:
            raise ValueError(f"Need at least 2 samples per class. Found class with only {min_class_size} samples")
        
        # Reshape X for CNN input (samples, timesteps, features, 1)
        X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
        print(f"Reshaped X to: {X.shape}")
        
        # Convert labels to categorical
        y_categorical = to_categorical(y, num_classes=len(actions))
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_categorical, test_size=0.2, random_state=42, stratify=y
        )
        
        print(f"\nTraining set: {X_train.shape}")
        print(f"Test set: {X_test.shape}")
        
        # Handle class imbalance
        class_weights = compute_class_weight(
            'balanced', 
            classes=np.unique(y), 
            y=y
        )
        class_weight_dict = dict(enumerate(class_weights))
        
        print(f"\nClass weights: {class_weight_dict}")
        
        # Create model
        model = create_hybrid_cnn_lstm_model(
            input_shape=(sequence_length, X.shape[2], 1), 
            num_classes=len(actions)
        )
        
        # Compile model
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        print(f"\nModel summary:")
        model.summary()
        
        # Create timestamp for TensorBoard logs
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        log_dir = f'logs/cricket_model_{timestamp}'
        
        # Callbacks
        callbacks = [
            TensorBoard(
                log_dir=log_dir,
                histogram_freq=1,
                write_graph=True,
                update_freq='epoch'
            ),
            EarlyStopping(patience=15, restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=10, min_lr=1e-7)
        ]
        
        print(f"\nTensorBoard logs will be saved to: {log_dir}")
        print(f"Run 'tensorboard --logdir={log_dir}' or use '%tensorboard --logdir {log_dir}' in Jupyter")
        
        # Train model
        print(f"\nStarting training...")
        history = model.fit(
            X_train, y_train,
            epochs=100,
            batch_size=16,
            validation_data=(X_test, y_test),
            callbacks=callbacks,
            class_weight=class_weight_dict,
            verbose=1
        )
        
        # Evaluate model
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"\nTest Accuracy: {test_accuracy:.4f}")
        
        # Save model  
        model.save('cricket_pose_model.h5')
        model.save('cricket_pose_model.keras')
        
        # Save label mapping for inference
        np.save('cricket_label_map.npy', label_map)
        
        print(f"\nModel and label map saved!")
        print(f"TensorBoard logs saved to: {log_dir}")
        
        return model, history, label_map
        
    except Exception as e:
        print(f"\nError in training pipeline: {e}")
        print("Please check your data and try again.")
        raise

In [62]:
# def real_time_cricket_prediction():
#     """Real-time cricket shot prediction"""
#     model = tf.keras.models.load_model('cricket_pose_model.h5')
#     label_map = np.load('cricket_label_map.npy', allow_pickle=True).item()
#     actions = list(label_map.keys())
    
#     # Prediction variables
#     sequence = []
#     predictions = []
#     threshold = 0.7
    
#     cap = cv.VideoCapture(0) 
    
#     with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         while cap.isOpened():
#             ret, frame = cap.read()
#             if not ret:
#                 break
            
#             # Make detections
#             image, results = mediapipe_detection(frame, holistic)
#             draw_styled_landmarks(image, results)
            
#             # Extract keypoints
#             keypoints = extract_keypoints(results)
#             sequence.append(keypoints)
#             sequence = sequence[-sequence_length:]
            
#             if len(sequence) == sequence_length:
#                 # Reshape for model input
#                 input_seq = np.expand_dims(np.array(sequence), axis=0)
#                 input_seq = input_seq.reshape(1, sequence_length, -1, 1)
                
#                 # Make prediction
#                 res = model.predict(input_seq, verbose=0)[0]
#                 predicted_action = actions[np.argmax(res)]
#                 confidence = np.max(res)
                
#                 # Display prediction
#                 if confidence > threshold:
#                     cv.putText(image, f'{predicted_action}: {confidence:.2f}', 
#                               (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
#                 # Visualization of probabilities
#                 for i, (action, prob) in enumerate(zip(actions, res)):
#                     y_pos = 100 + i * 30
#                     cv.rectangle(image, (10, y_pos), (int(prob * 300) + 10, y_pos + 25), 
#                                (0, 255, 0), -1)
#                     cv.putText(image, f'{action}: {prob:.2f}', (15, y_pos + 18), 
#                               cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
#             cv.imshow('Cricket Pose Estimation', image)
            
#             if cv.waitKey(10) & 0xFF == ord('q'):
#                 break
    
#     cap.release()
#     cv.destroyAllWindows()


In [63]:
def clear_processed_data():
    """Clear all processed data to force reprocessing"""
    import shutil
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH)
        print(f"Cleared processed data directory: {PROCESSED_DATA_PATH}")
    else:
        print("No processed data directory found to clear.")

if __name__ == "__main__":
    print("Cricket Pose Estimation with Enhanced Debug and TensorBoard")
    print("="*60)
    
    # Check data structure first
    if not check_data_structure():
        print("Please fix the data structure issues and try again.")
        exit(1)
    
    # Options for running
    print("\nOptions:")
    print("1. Train model (use existing processed sequences if available)")
    print("2. Train model (force reprocess all videos)")
    print("3. Clear processed data")
    print("4. Real-time prediction")
    
    choice = input("\nEnter your choice (1-4): ").strip()
    
    try:
        if choice == '1':
            model, history, label_map = train_cricket_model(force_reprocess=False)
        elif choice == '2':
            model, history, label_map = train_cricket_model(force_reprocess=True)
        elif choice == '3':
            clear_processed_data()
        elif choice == '4':
            real_time_cricket_prediction()
        else:
            print("Invalid choice. Running default training...")
            model, history, label_map = train_cricket_model(force_reprocess=False)
    except Exception as e:
        print(f"Program terminated with error: {e}")
        print("Please check the error messages above and fix any issues.")

Cricket Pose Estimation with Enhanced Debug and TensorBoard
Checking data structure...
Sweep: 27 videos
FBD: 15 videos
Cover drive: 29 videos
Reverse Sweep: 30 videos
loft: 31 videos
Straight Drive: 25 videos
Backfoot punch: 19 videos
Cut Shot: 43 videos
Flick: 22 videos
Front Food defence: 32 videos
On Drive: 40 videos
Uppercut: 29 videos
Pull Shot: 40 videos
Total videos found: 382

Options:
1. Train model (use existing processed sequences if available)
2. Train model (force reprocess all videos)
3. Clear processed data
4. Real-time prediction
Checking data structure...
Sweep: 27 videos
FBD: 15 videos
Cover drive: 29 videos
Reverse Sweep: 30 videos
loft: 31 videos
Straight Drive: 25 videos
Backfoot punch: 19 videos
Cut Shot: 43 videos
Flick: 22 videos
Front Food defence: 32 videos
On Drive: 40 videos
Uppercut: 29 videos
Pull Shot: 40 videos
Total videos found: 382
Detected cricket shots: ['Backfoot punch' 'Cover drive' 'Cut Shot' 'FBD' 'Flick'
 'Front Food defence' 'On Drive' 'Pull S

I0000 00:00:1753627614.350877    7643 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1753627614.406537   12658 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2
W0000 00:00:1753627614.438845   12631 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753627614.457904   12645 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753627614.458884   12640 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753627614.459289   12634 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000

    Extracted 6 sequences from VID_20250723_115632159_00001220.mov
  Processing video 2/19: VID_20250723_115632159_00002115.mov
    Extracted 5 sequences from VID_20250723_115632159_00002115.mov
  Processing video 3/19: VID_20250723_115632159_00000905.mov
    Extracted 6 sequences from VID_20250723_115632159_00000905.mov
  Processing video 4/19: VID_20250723_115632159_00000735.mov
    Extracted 6 sequences from VID_20250723_115632159_00000735.mov
  Processing video 5/19: VID_20250723_115632159_00002435.mov
    Extracted 6 sequences from VID_20250723_115632159_00002435.mov
  Processing video 6/19: VID_20250723_115632159_00002600.mov
    Extracted 6 sequences from VID_20250723_115632159_00002600.mov
  Processing video 7/19: VID_20250723_115632159_00000555.mov
    Extracted 5 sequences from VID_20250723_115632159_00000555.mov
  Processing video 8/19: VID_20250723_115632159_00000125.mov
    Extracted 6 sequences from VID_20250723_115632159_00000125.mov
  Processing video 9/19: VID_20250723

KeyboardInterrupt: 